### Методика подготовки кастомизированного Allowed List на основе файла от Кастомера
С помощью данной методики можно:
- Подготовить команды для загрузки Combined Allowed листов для Кастомеров <br>
- Подготовить команды для загрузки STS листов для Кастомеров <br>

In [30]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join, normpath
import os
import roamability as rb
import re
from collections import namedtuple

#######################################################################################
# Define variables here
#######################################################################################

# Input files
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Sat_Jul_25_2020_after.csv'
dmi_sts_list = 'Steering.sts_sponsor_country_export_Tue_Jun_02_2020.csv'

max_data_rate_to_consired_discounted = 0.7

file_new_al = 'AL_TOTTOLI_S8.csv' # File name with the new Allowed list
# The example of the content of the file with the list of allowed networks:
# TADIG,SPONSOR
# ALBVF,S1
# ARGCM,S5
# ARM01,S2
# AUSOP,S1
# AUTHU,S2

# Output files
out_file_not_disc = 'AlNotDiscounted.csv'
out_file_toc_al_delete_not_disc = 'AlNotDiscountedDelete.rcjson'

# Задать имена для создаваемых, кастомизированных Allowed list
# Правило задания имен: <ИмяСпоносра>_TEST_COMB
s1_al_name = ''
s2_al_name = ''
s4_al_name = 'MB_Tottolli'
s5_al_name = 'SMART_Tottol'
s6_al_name = ''
s8_al_name = 'S8_Tottolli'

# Задать имена для создаваемых, кастомизированных [sts] Sponsor Profile
# Правило задания имен: S1_TEST_STS
s1_sts_profile_name = 'S1_SIMT_STS'
s2_sts_profile_name = 'S2_SIMT_STS'
s4_sts_profile_name = 'S4_SIMT_STS'
s5_sts_profile_name = ''
s6_sts_profile_name = ''
s8_sts_profile_name = ''

# Задать имена для существующих "All" Allowed list (разрешены будут только те сети, которые присутствуют в 'All')
s1_al_all_name = 'Partner_All'
s2_al_all_name = 'P4_All'
s4_al_all_name = 'MB_ALL'
s5_al_all_name = 'SMART'
s6_al_all_name = 'S6_ALL'
s8_al_all_name = 'S8_ALL'

# Задать имена для существующих, основных "Combined" Allowed list
s1_al_combined_def_name = 'Partner_Combined'
s2_al_combined_def_name = 'P4_Combined'
s4_al_combined_def_name = 'MB_Combined'
s5_al_combined_def_name = 'SMART_Combined'
s6_al_combined_def_name = 'S6_Combined'
s8_al_combined_def_name = 'P8_Combined'

#######################################################################################

#######################################################################################

sponsor_object = namedtuple('Sponsor',
                'sponsor sponsor_id al_all_name al_pattern al_name_combined_custom al_name_combined_def sts_profile_name')

s1_sponsor = sponsor_object('S1', 1, s1_al_all_name, r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*',
                            s1_al_name, s1_al_combined_def_name, s1_sts_profile_name)
s2_sponsor = sponsor_object('S2', 2, s2_al_all_name, r'[Pp]4[\s\w-]*|^S2_[\s\w-]*',
                            s2_al_name, s2_al_combined_def_name, s2_sts_profile_name)
s4_sponsor = sponsor_object('S4', 4, s4_al_all_name, r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*',
                            s4_al_name, s4_al_combined_def_name, s4_sts_profile_name)
s5_sponsor = sponsor_object('S5', 5, s5_al_all_name, r'[Ss][Mm][Aa][Rr][Tt][\s\w-]*|^S5_[\s\w-]*',
                            s5_al_name, s5_al_combined_def_name, s5_sts_profile_name)
s6_sponsor = sponsor_object('S6', 6, s6_al_all_name, r'^S6_[\s\w-]*',
                            s6_al_name, s6_al_combined_def_name, s6_sts_profile_name)
s8_sponsor = sponsor_object('S8', 8, s8_al_all_name, r'^S8_[\s\w-]*',
                            s8_al_name, s8_al_combined_def_name, s8_sts_profile_name)

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

def prepare_toc_to_create_al(df, downloads, result_file_name):
    """df[['SPONSOR','TADIG_AL','MOC','3G_ONLY']], where SPONSOR - S1, S2..."""
    df = df[['SPONSOR','TADIG_AL','MOC','3G_ONLY']].dropna().drop_duplicates()
    df.MOC = df.MOC.astype('int')
    df[['3G_ONLY']] = df[['3G_ONLY']].astype('int')
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1 = '{"params":{"sponsor":"'
    t2 = '","plmnCode":"'
    t3 = '","mo":"'
    t4 = '","mt":"1","sms":"1","data":"'
    t5 = '","moSec":"1","dataKb":"1"},"type":"create","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig, moc, only_3g] in enumerate(df.values):
        for [sponsor, al_name_combined] in [[sponsor.sponsor, sponsor.al_name_combined_custom] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + al_name_combined + t2 + tadig + t3 + str(moc) + t4 + str(only_3g) + t5)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_al(df, downloads, result_file_name):
    """df[['Sponsor','TADIG']], where Sponsor - Allowed list name like Partner_Combined, P4_Combined..."""
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsor":"'
    t2='","plmnCode":"'
    t3='"},"type":"remove","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig] in enumerate(df.values):
        ouf.write(t1 + sponsor_name + t2 + tadig + t3)
        if i < num_val:
            ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_create_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0]-1
    t1 = '{"params":{"sponsorProfile":"'
    t2 = '","countryName":"'
    t3 = '","mcc":"'
    t4 = '"},"type":"create","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsorProfile":"'
    t2='","countryName":"'
    t3='","mcc":"'
    t4='"},"type":"remove","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def check_al_for_differences(df_al_current, df_al_new):
    """df_al_current[['Sponsor','TADIG', 'SPONSOR']], df_al_new[['Sponsor','TADIG', 'SPONSOR','MOC','3G_ONLY']]"""
    df = pd.merge(df_al_current[['Sponsor','TADIG','SPONSOR']],
                    df_al_new[['Sponsor','TADIG', 'SPONSOR','MOC','3G_ONLY']], how='outer',
                    on='TADIG', suffixes=['_CURRENT', '_NEW'])
    return df[(df.SPONSOR_CURRENT != df.SPONSOR_NEW)]

In [31]:
# Получение данных TADIG
# Получение прайсов с флагом is_discounted = 1

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE AS TADIG
,mcc.MCC
,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mcc ON mcc.network_id = n.network_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

###################################################
# Adjustments
df_tadig.drop(
    df_tadig[(df_tadig.TADIG=='YUGPM') & (df_tadig.NETWORK_ID==623)].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################
    
sql_srt=\
'''
SELECT r.NETWORK_ID
,r.SPONSOR_ID AS SPONSOR
,r.data_rate
FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329,343,368,372)
--AND ((r.start_date < GETDATE() AND r.end_date is null) OR (r.start_date < GETDATE() AND r.end_date > GETDATE()))
AND r.end_date IS NULL
AND r.is_discounted = 1
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_disc = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
print('TADIGs and MCC with NETWORK_ID:')
display(df_tadig.head(3))

replace_sponsor_id_dict = dict((sponsor.sponsor_id, sponsor.sponsor) for sponsor in sponsors)
df_disc.SPONSOR = df_disc.SPONSOR.replace(replace_sponsor_id_dict)

print('\nHigh cost in Discounted Networks:')
print('!!!WILL BE REMOVED FROM ALLOWED LIST!!!')
display(df_disc[df_disc.data_rate > max_data_rate_to_consired_discounted])

# Remove high data priced networks
df_disc = df_disc[df_disc.data_rate <= max_data_rate_to_consired_discounted]

TADIGs and MCC with NETWORK_ID:


,COUNTRY_NAME,NETWORK_NAME,TADIG,MCC,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,412,4
1,Afghanistan,Areeba/MTN,AFGAR,412,5
2,Afghanistan,Etisalat,AFG55,412,6



High cost in Discounted Networks:
!!!WILL BE REMOVED FROM ALLOWED LIST!!!


,NETWORK_ID,SPONSOR,data_rate


In [32]:
# Create df for existing "All" allowed lists 
# Create df for new "Allowed list"

# Like expression for Allowed List

df_al = pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code','MO','Only 3G allowed'])
df_al.rename({'PLMN code':'TADIG','MO':'MOC','Only 3G allowed':'3G_ONLY'}, axis=1, inplace=True)

replace_dict = dict((sponsor.al_pattern, sponsor.sponsor) for sponsor in sponsors)

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace=replace_dict, regex=True)

df_al = pd.merge(df_al, df_tadig, how='left', on='TADIG')

# Check not found Allowed Lists

print('Not found Allowed Lists:\n')
for sponsor in df_al.SPONSOR.unique().tolist():
    if sponsor not in [sponsor.sponsor for sponsor in sponsors]:
        print(sponsor)
print('-' * 80)

sponsor_cols = ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG','MOC','3G_ONLY']

df_al_s1 = df_al.loc[df_al.Sponsor.isin([s1_sponsor.al_all_name]), sponsor_cols]
df_al_s2 = df_al.loc[df_al.Sponsor.isin([s2_sponsor.al_all_name]), sponsor_cols]
df_al_s4 = df_al.loc[df_al.Sponsor.isin([s4_sponsor.al_all_name]), sponsor_cols]
df_al_s5 = df_al.loc[df_al.Sponsor.isin([s5_sponsor.al_all_name]), sponsor_cols]
df_al_s6 = df_al.loc[df_al.Sponsor.isin([s6_sponsor.al_all_name]), sponsor_cols]
df_al_s8 = df_al.loc[df_al.Sponsor.isin([s8_sponsor.al_all_name]), sponsor_cols]

df_al_s_list = [df_al_s1, df_al_s2, df_al_s4, df_al_s5, df_al_s6, df_al_s8]

print('\nAll Allowed lists (heads):')
for df_temp in df_al_s_list:
    display(df_temp.head(3))

df_new_al = pd.read_csv(join(downloads, file_new_al), sep=',')
df_new_al = pd.merge(df_new_al, df_tadig, how='left', on='TADIG')
df_new_al.drop_duplicates(inplace=True)
print('\nThe new allowed list uploaded from file enriched by MCC and NETWORK_ID:')
df_new_al.head(3)

Not found Allowed Lists:

--------------------------------------------------------------------------------

All Allowed lists (heads):


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
50,Partner_All,S1,769,RUSNW,1,0
2572,Partner_All,S1,12,ALBVF,1,0
2573,Partner_All,S1,30,ARM01,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
90,P4_All,S2,10,ALBEM,1,0
211,P4_All,S2,1024,GBRME,1,0
627,P4_All,S2,1025,GBRVF,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
4266,MB_ALL,S4,49,AUSVF,1,0
4268,MB_ALL,S4,199,CHNCU,1,0
4269,MB_ALL,S4,392,HKGH3,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
233,SMART,S5,12,ALBVF,0,0
638,SMART,S5,25,ARGCM,0,0
708,SMART,S5,51,AUTTR,0,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
704,S6_ALL,S6,433,IDN89,1,0
1294,S6_ALL,S6,769,RUSNW,1,0
1359,S6_ALL,S6,770,RUS01,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
542,S8_ALL,S8,770,RUS01,1,0
637,S8_ALL,S8,20,AIACW,1,0
1703,S8_ALL,S8,10,ALBEM,1,0



The new allowed list uploaded from file enriched by MCC and NETWORK_ID:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID
0,ABWDC,S8,Aruba,New millennium Telcom services,363,1160
1,AFGEA,S8,Afghanistan,Etisalat,412,6
2,AIACW,S8,Anguilla,Cable and Wireless,365,20


In [33]:
# Check mismatches with 'ALL' Allowed Lists in DMI and the new Customized Combined Allowed List
# Prepare a file with TADIGs for creating Allowed List

def check_allowed(sponsor, df_new, df_allowed):
    df = pd.merge(df_new[df_new.SPONSOR == sponsor], df_allowed, how='left', on=['NETWORK_ID','SPONSOR'],
                      suffixes=['', '_AL'])
    return df

df_al_new_create = DataFrame()

for sponsor, df_al_s in zip(sponsors, df_al_s_list):
    print(f'Missing in {sponsor.al_all_name} Allowed list in DMI')
    df_temp_al = check_allowed(sponsor.sponsor, df_new_al, df_al_s)
    df_temp_missing = df_temp_al[df_temp_al.Sponsor.isnull()]
    display(df_temp_missing)
    df_temp_missing.to_csv(join(downloads, f'{sponsor.sponsor.lower()}_al_missing.csv'),index=False)
    
    df_temp_al.dropna(subset=['TADIG_AL'], inplace=True)
    df_temp_al.MOC = df_temp_al.MOC.astype('int64')
    df_temp_al['3G_ONLY'] = df_temp_al['3G_ONLY'].astype('int64')
    
    df_al_new_create = df_al_new_create.append(df_temp_al, ignore_index=True, sort=False)

print(f'Create new Allowed list in DMI')
df_al_new_create.to_csv(join(downloads, f'df_al_create.csv'), index=False)
df_al_new_create.head(3)

Missing in Partner_All Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL,MOC,3G_ONLY


Missing in P4_All Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL,MOC,3G_ONLY


Missing in MB_ALL Allowed list in DMI


...

,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL,MOC,3G_ONLY


Missing in S8_ALL Allowed list in DMI


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,Sponsor,TADIG_AL,MOC,3G_ONLY


Create new Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL,MOC,3G_ONLY
0,ABWDC,Aruba,New millennium Telcom services,363,S8_ALL,S8,1160,ABWDC,1,0
1,AFGEA,Afghanistan,Etisalat,412,S8_ALL,S8,6,AFGEA,1,0
2,AIACW,Anguilla,Cable and Wireless,365,S8_ALL,S8,20,AIACW,1,0


In [20]:
# Check for different Sponsors in the same country

s1_temp = df_new_al.groupby('COUNTRY_NAME')['SPONSOR'].nunique()
s1_temp[s1_temp.values > 1]

Series([], Name: SPONSOR, dtype: int64)

In [21]:
# Check for not discounted in the new Allowed list

df_temp = pd.merge(df_new_al, df_disc, how='left', on=['NETWORK_ID', 'SPONSOR'])
df_temp[df_temp.data_rate.isnull()]

,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate


### Prepare the TOC commands to create the full new Allowed list for Customer

In [22]:
# Create file with TOC commands to create the full new Allowed list for Customer

result_file_name_al_create = 'al_create.rcjson' # file name with TOC comands for cerating the new Allowed list
df_temp = df_al_new_create[['SPONSOR','TADIG_AL','MOC','3G_ONLY']].dropna().drop_duplicates()
prepare_toc_to_create_al(df_temp, downloads, result_file_name_al_create)

### Check for the inconsystency with the Customer's Combined

In [7]:
# Проверить различия с существующим Allowed list Кастомера.
# Проверяются только те операторы, которые были найдены в "All" Allowed list спонсоров.

df_al_custom_current = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]),
                           ['Sponsor', 'TADIG', 'SPONSOR']]
df_al_custom_new = df_al_new_create[['Sponsor','SPONSOR','TADIG_AL','MOC','3G_ONLY']].rename({'TADIG_AL':'TADIG'}, axis=1).dropna().drop_duplicates()
df_make_changes_custom_current = check_al_for_differences(df_al_custom_current, df_al_custom_new)
df_make_changes_custom_current[['TADIG','SPONSOR_CURRENT','SPONSOR_NEW']].sort_values(by='TADIG')

,TADIG,SPONSOR_CURRENT,SPONSOR_NEW
0,AUSVF,NaN,S4
1,CHNCU,NaN,S4
2,EGYK9,NaN,S4
3,HKGH3,NaN,S4
4,IDNSL,NaN,S4
5,ISRPL,NaN,S4
6,JPNDO,NaN,S4
7,KORKF,NaN,S4
8,MACHT,NaN,S4
9,MYSMI,NaN,S4


### Check for the inconsystency with the default Combined

In [8]:
# Проверть, что нет несоответствия с Combined
# !!! после загрузки новых Allowed list

df_al_comb = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_def for sponsor in sponsors]),
                       ['Sponsor','TADIG', 'SPONSOR']]
df_al_customer = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]),
                           ['Sponsor','TADIG', 'SPONSOR','MOC','3G_ONLY']]
df_changes_comb_def = check_al_for_differences(df_al_comb, df_al_customer)
df_changes_comb_def[['TADIG','SPONSOR_CURRENT','SPONSOR_NEW']].sort_values(by='TADIG')

,TADIG,SPONSOR_CURRENT,SPONSOR_NEW
110,AIACW,S1,NaN
49,ALBVF,S1,NaN
111,ARG01,S1,NaN
50,ARM01,S1,NaN
201,ARM05,S1,NaN
...,...,...,...
150,VCTCW,S1,NaN
151,VGBCW,S1,NaN
108,VNMVT,S1,NaN
101,ZAFVC,S1,NaN


### Prepare the TOC commands to adjust Allowed list for Customer

In [10]:
# Prepare commands to adjust the existing AL of customer by deleting the extras

delete_file_name = 'al_adjust_delete.rcjson' # file name with TOC comands for deleting the extras
df_temp = df_make_changes_custom_current[['Sponsor_CURRENT','TADIG']].dropna().drop_duplicates()
prepare_toc_to_delete_al(df_temp, downloads, delete_file_name)

In [11]:
# Prepare commands to adjust the existing AL of customer by createing missing operators in Allowed list

create_file_name = 'al_adjust_create.rcjson' # file name with TOC comands for creating missing operators in Allowed list
df_temp = df_make_changes_custom_current[['SPONSOR_NEW','TADIG','MOC','3G_ONLY']].rename({
    'SPONSOR_NEW':'SPONSOR', 'TADIG':'TADIG_AL'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_create_al(df_temp, downloads, create_file_name)

### Prepare the TOC commands to create full new STS list for Customer

In [12]:
# Создать новые правила STS с нуля

result_file_name_sts_create = 'sts_create.rcjson'  # TOC comands for cerating STS list
df_temp = df_al_new_create[['SPONSOR','MCC','COUNTRY_NAME']].dropna().drop_duplicates()
prepare_toc_to_create_sts(df_temp, downloads, result_file_name_sts_create)

### Prepare the TOC commands to adjust STS list for Customer

In [13]:
# Проверить различия с существующим STS list Кастомера.

df_sts = pd.read_csv(join(downloads, dmi_sts_list))
df_sts.drop('Country name', axis=1, inplace=True)
df_sts.rename({'Sponsor profile':'STS_PROFILE_NAME'}, axis=1, inplace=True)

df_sts['SPONSOR'] = df_sts['STS_PROFILE_NAME'].replace(to_replace=replace_dict, regex=True)

df_sts_new = df_al_new_create[['SPONSOR','MCC','COUNTRY_NAME']].dropna().drop_duplicates()
df_sts_new.MCC = df_sts_new.MCC.astype('int64')

df_sts_customer = df_sts.loc[df_sts.STS_PROFILE_NAME.isin([s1_sponsor.sts_profile_name,
                                               s2_sponsor.sts_profile_name,
                                               s4_sponsor.sts_profile_name,
                                               s5_sponsor.sts_profile_name])]

df_sts_customer = pd.merge(df_sts_customer,
                 df_sts_new, how='outer', on='MCC',
                suffixes=['_CUSTOM_CURRENT', '_CUSTOM_NEW'])

df_make_changes = df_sts_customer[(df_sts_customer.SPONSOR_CUSTOM_CURRENT != df_sts_customer.SPONSOR_CUSTOM_NEW)]
df_make_changes

,STS_PROFILE_NAME,SPONSOR_CUSTOM_CURRENT,SPONSOR_CUSTOM_NEW,MCC,COUNTRY_NAME
0,NaN,NaN,S1,250,Russian Federation
1,NaN,NaN,S1,640,Tanzania
2,NaN,NaN,S1,276,Albania
3,NaN,NaN,S1,283,Armenia
4,NaN,NaN,S1,505,Australia
...,...,...,...,...,...
114,NaN,NaN,S2,297,Montenegro
115,NaN,NaN,S4,520,Thailand
116,NaN,NaN,S4,420,Saudi Arabia
117,NaN,NaN,S4,455,Macao China


In [14]:
# Prepare commands to adjust the existing STS list by deleting the extras

delete_file_name = 'sts_adjust_delete.rcjson'
df_temp = df_make_changes[['SPONSOR_CUSTOM_CURRENT','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_CURRENT':'SPONSOR'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_delete_sts(df_temp, downloads, delete_file_name)

In [15]:
# Prepare commands to create missing countries in STS list

create_file_name = 'sts_adjust_create.rcjson'
df_temp = df_make_changes[['SPONSOR_CUSTOM_NEW','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_NEW':'SPONSOR'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_create_sts(df_temp, downloads, create_file_name)

### После обновления AL обновить MCC-MAP файл

### Create SX_ALL allowed list

In [68]:
#-----------------------------------------
# Set variables here

sponsor = 'S8'

al_all_file = 's8_al_missing.csv'
# The example of the content of the file with the list of allowed networks:
# TADIG,SPONSOR,NETWORK_ID
# ALBVF,S1,111
# ARGCM,S5,222
# ARM01,S2,333

output_file_csv_with_missing = 'df_al_create.csv'
result_file_name_al_create = 'al_create.rcjson' # file name with TOC comands for cerating the new Allowed list

#-----------------------------------------

# Download data from file

df_missing = pd.read_csv(join(downloads, al_all_file))

# Select only discounted

disc_list = df_disc.loc[df_disc.SPONSOR == sponsor, 'NETWORK_ID'].to_numpy(dtype='int64')
df_missing = df_missing.loc[df_missing.NETWORK_ID.isin(disc_list)]

# Select TADIGs from all ALL allowed lists of all sponsors

df_tadig = DataFrame()
for df in df_al_s_list:
    df_tadig = df_tadig.append(df[['TADIG','NETWORK_ID']])
    df_tadig.drop_duplicates(inplace=True)

# Prepare the DataFrame for creating the allowed list

df_al_new_create = pd.merge(df_missing, df_tadig,
                            how='left', on='NETWORK_ID', suffixes=['', '_AL'])[['SPONSOR','TADIG_AL']]

df_al_new_create['MOC'] = 1
df_al_new_create['3G_ONLY'] = 0
df_al_new_create.dropna(inplace=True)
# df_al_new_create.rename({'TADIG_AL':'TADIG'}, axis=1, inplace=True)
df_al_new_create.to_csv(join(downloads, output_file_csv_with_missing), index=False)


# Prepare TOC commands to add missing TADIGs
prepare_toc_to_create_al(df_al_new_create, downloads, result_file_name_al_create)

print('Missing:')
display(df_al_new_create)

Missing:


,SPONSOR,TADIG_AL,TADIG_AL,MOC,3G_ONLY


### Find Not Discounted in any Allowed Lists in DMI

In [34]:
# Find Not Discounted in any Allowed Lists in DMI and prepare TOC commands to delete

df_temp = pd.merge(df_al, df_disc, how='left', on=['NETWORK_ID','SPONSOR'])
df_temp = df_temp[df_temp.data_rate.isnull()]
df_temp.to_csv(join(downloads, out_file_not_disc), index=False)

# Prepare commands to adjust the existing AL of customer by deleting the extras
prepare_toc_to_delete_al(df_temp[['Sponsor','TADIG']].dropna().drop_duplicates(), downloads, out_file_toc_al_delete_not_disc)

df_temp

,Sponsor,TADIG,MOC,3G_ONLY,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate


### Developments